# AuxTel Calibration Illumination System Lamp Warm-up test

Craig Lage - 09Feb23

In [ ]:
import asyncio
import sys, time, os
from lsst.ts import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display

from astropy.io import fits
from astropy.time import Time, TimeDelta
from astropy.table import Table

from lsst_efd_client import EfdClient
client = EfdClient('summit_efd')

from lsst.ts import salobj

In [ ]:
# change this to a local directory
data_dir = '/home/craiglagegit/DATA'

In [ ]:
d = salobj.Domain()

### Connect to CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

In [ ]:
await atmonochromator.start_task
await FiberSpectrograph.start_task
await electrometer.start_task
await WhiteLightSource.start_task

In [ ]:
# Check that the monochromator is in white light mode - grating 2, wavelength 0
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

# functions to run the electromter measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    return filename, lfa

In [ ]:
# Run electrometer calibration
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)    

In [ ]:
#Start chiller
tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
tmp = await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

### Test Electrometer

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

In [ ]:
# To change range.  -1 is autorange
await electrometer.cmd_setRange.set_start(setRange=-1)

### Check monochromator state
### Assumes ATWhiteLight, Electrometer, and Monochromator are enabled and working

In [ ]:
# This test was done with white light
await get_params()

In [ ]:
#Turn on lamp and continue readings.  Then continue for ~ 1 hour

#readings_on = []
fig = plt.figure(figsize = (4,4))
for i in range(200):
    plt.cla()
    if i == 2:
        tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power = 910)
    start_time = Time(time.time(),format='unix', scale='utc')
    exp_time = 20 #sec
    filename, lfa = await elec_meas(exp_time)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    for (t,current) in data:
        obs_time = start_time + TimeDelta(t, format='sec')
        readings_on.append([obs_time.unix_tai, -current])
    
    plot_stuff = np.array(readings_on)
    display.clear_output(wait=True)
    plt.plot(plot_stuff[:,0]-plot_stuff[0,0], plot_stuff[:,1])
    plt.ylim(-0.1E-5,5E-5)
    display.display(plt.gcf())



In [ ]:
#Turn lamp power up and continue readings.  Then continue for ~ 30 minutes

fig = plt.figure(figsize = (4,4))
for i in range(100):
    plt.cla()
    if i == 2:
        tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=1000)
    start_time = Time(time.time(),format='unix', scale='utc')
    exp_time = 20 #sec
    filename, lfa = await elec_meas(exp_time)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    for (t,current) in data:
        obs_time = start_time + TimeDelta(t, format='sec')
        readings_on.append([obs_time.unix_tai, -current])
    
    plot_stuff = np.array(readings_on)
    display.clear_output(wait=True)
    plt.plot(plot_stuff[:,0]-plot_stuff[0,0], plot_stuff[:,1])
    plt.ylim(-0.1E-5,5E-5)
    display.display(plt.gcf())



In [ ]:
fig = plt.figure(figsize = (4,4))
plt.title("WhiteLight Lamp Stability study - 14Feb23")
plt.plot(plot_stuff[:,0]-plot_stuff[0,0], plot_stuff[:,1])
plt.ylim(-0.1E-5,4E-5)
plt.plot([0.0,0.0],[2.0E-5,2.5E-5], color='black')
plt.text(50, 2.5E-5, "Lamp on 920 W")
plt.plot([4600.0,4600.0],[2.0E-5,2.5E-5], color='black')
plt.text(3000, 1.8E-5, "Lamp up to 1000 W")
plt.plot([5400.0,5400.0],[3.0E-5,3.5E-5], color='black')
plt.text(4000, 3.6E-5, "Lamp faulted")
plt.xlabel("Time(sec)")
plt.ylabel("Current (Amps)")
plt.savefig(f"/home/craiglagegit/DATA/WhiteLight_WarmUp_14Feb23.pdf")